# Object Tracking

In [1]:
import cv2
import numpy as np
import math
import glob
import os
import copy
from tqdm import tqdm
from matplotlib import pyplot as plt

from utils import *

# Lucas Kanade Tracker Implementation

In [8]:
def getDeltaP(b, gradImage, templatePts):
    
    jacobian = getJacobian(templatePts)
    gradImage = np.expand_dims(gradImage, axis=1)

    A = np.matmul(gradImage, jacobian)
    A = np.squeeze(A, axis=1)

    Hessian = np.matmul(A.T, A)
    AT_b = np.matmul(A.T, b)

    deltaP = np.matmul(np.linalg.pinv(Hessian), AT_b)

    return deltaP

In [9]:
def lucasKanadeTracker(templatePts, templateIntensity, Image, p):
    
    threshold = 0.035
    maxIter = 150
    deltaP_norm = np.inf

    gradientX = cv2.Sobel(Image,cv2.CV_64F,1,0,ksize=3)
    gradientY = cv2.Sobel(Image,cv2.CV_64F,0,1,ksize=3)

    for i in range(maxIter):

        W = getWarpMat(p)
        warpedImgPts, warpedImgIntensity, gradXValues, gradYValues = warpPoints(Image, W, templatePts, gradientX, gradientY)

        error = templateIntensity - warpedImgIntensity
        
        gradImage = np.stack((gradXValues, gradYValues), axis=1)
        
        deltaP = getDeltaP(error, gradImage, templatePts)
        deltaP_norm = np.linalg.norm(deltaP)

        p += deltaP

        if (deltaP_norm < threshold):
            break

    return W,p

In [10]:
def trackObject(frames, ROI):
    
    tracker_output = []
    roi_Pt1, roi_Pt2 = np.array([ROI[0], ROI[1], 1]), np.array([ROI[0]+ROI[2], ROI[1]+ROI[3], 1])

    template = frames[...,0]
    templatePts, templateIntensity = getUnrolledPts(template, ROI)
    mean_template = np.mean(template)

    p = np.zeros(6, dtype = float)

    for i in tqdm(range(1, frames.shape[-1])):
        
        frame_current = copy.deepcopy(frames[...,i])
        frame_display = frame_current.copy()
        
        # Normalizing the intensity for new image with respect to template image
        mean_frame = np.mean(frame_current)
        frame_current = frame_current.astype(float)
        frame_current = frame_current *(mean_template/mean_frame)

        # Lucas Kanade Tracker
        warpMat, p = lucasKanadeTracker(templatePts, templateIntensity, frame_current, p)

        roi_Pt1_new = np.matmul(warpMat, roi_Pt1).astype(int)
        roi_Pt2_new = np.matmul(warpMat, roi_Pt2).astype(int)

        cv2.rectangle(frame_display, tuple(roi_Pt1_new), tuple(roi_Pt2_new), (0, 0, 255), 2)
        tracker_output.append(frame_display)

    return tracker_output

# Loading Data

In [11]:
data_dir = '../Data/'
videos = {}

for vid in os.listdir(data_dir):
    videos[vid.split('.')[0]] = np.load(data_dir + vid)

print('Videos Loaded: ', list(videos.keys()))

Videos Loaded:  ['car1', 'car2', 'landing']


In [12]:
play_video(videos['car1'], type='array')

Playing video ....
Press q to exit


# Applying Tracker

## Car_1

In [13]:
selection = 'car1'
frames = videos[selection]

In [14]:
# ROI = get_rectangle(frames[...,0])
ROI = [118, 103, 226, 179]
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  [118, 103, 226, 179]


In [15]:
tracker_car1 = trackObject(frames, ROI)

100%|██████████| 260/260 [00:41<00:00,  6.21it/s]


In [17]:
play_video(tracker_car1, type='list')

Playing video ....
Press q to exit


## Car_2

In [18]:
selection = 'car2'
frames = videos[selection]

In [19]:
ROI = get_rectangle(frames[...,0])
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  [37, 97, 128, 70]


In [20]:
tracker_car2 = trackObject(frames, ROI)

100%|██████████| 414/414 [00:31<00:00, 12.99it/s]


In [21]:
play_video(tracker_car2, type='list')

Playing video ....
Press q to exit


## Landing

In [22]:
selection = 'landing'
frames = videos[selection]

In [23]:
ROI = get_rectangle(frames[...,0])
# ROI = [426, 78, 135, 59]
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  [426, 78, 135, 59]


In [24]:
tracker_landing = trackObject(frames, ROI)

100%|██████████| 49/49 [00:03<00:00, 12.69it/s]


In [25]:
play_video(tracker_landing, type='list')

Playing video ....
Press q to exit
